# 2nd session - trace alignment

made by 임정은

- trace alignment 기능을 만들어 보려고 합니다.
- trace alignment는 이벤트 로그들을 좀 더 쉽게 파악하기 위해 trace들을 잘 정렬해서 시각화해주는 것으로, 이미 도출된 프로세스 정보를 이용하거나, 그냥 raw traces 정보를 이용합니다.
- trace alignment를 이용하는 이유는 그냥 process discovery만 하고 땡 하는 것보다 trace들을 시각화해서 보여주면 아래의 질문에 좀 더 잘 답할 수 있기 때문입니다.
    - 가장 흔한 process behavior은?
    - 주어진 프로세스의 deviation이 일어나는 곳은 어디이며, deviation이 발생하는 프로세스들의 공통점은?
    - 프로세스에서 주요 공통 패턴은?
    - 어떤 activity나 set of activity가 발생하는 contexts는?
    - 어떤 behavior가 나타나거나, 그와 비슷한 behavior가 나타나는 프로세스 instances에는 뭐가 있나?
- 참고: http://www.ormbunkar.se/aliview/

![trace alignment example](https://www.researchgate.net/profile/Hicheur_Awatef/publication/281104380/figure/fig1/AS:284519203655692@1444846072767/Figure-2-Trace-alignment-of-traces-in-the-training-event-log-example-for-one-of-the.png)

## required library

- 'tkinter' : GUI를 간단하게 표현할 수 있는 모듈로, 파이썬 설치시 기본적으로 내장되어 있는 파이썬 표준 라이브러리이다. (그래서 다른 GUI 프레임워크나 툴킷에 비해 지원되는 위젯들이 부족하고 UI도 별로 안 예쁘다고 함)
- 다른건 뭐가 더 필요할지 아직 잘 모르겠습니다..

In [2]:
import pandas as pd
import numpy as np
import random
from tkinter import *
import sys

'''%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import networkx as nx
import itertools as ittls
import functools 
import time'''

'%matplotlib inline\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nimport datetime as dt\nimport networkx as nx\nimport itertools as ittls\nimport functools \nimport time'

## Example
- 우선 2개의 trace를 비교할 때의 예를 구현해보겠습니다

In [14]:
T1 = ['A', 'B', 'C', 'N', 'J', 'R', 'O', 'C', 'L', 'C', 'R', 'P', 'M']
T2 = ['A', 'J', 'C', 'J', 'N', 'R', 'C', 'K', 'C', 'R', 'B', 'P']

## Matrix form
- maximum-match를 찾을 때 사용할 matrix 틀
- 첫 행,열은 틀을 만들 때 미리 채워지도록 했습니다
    - gap에 penalty를 얼마나 줄 것인지에 따라서 값이 달라질 수 있음
- tr_matrix는 각 cell까지 alignment를 했을 때 최대 점수를 dataframe 형태로,
- previnfo_matrix는 각 최대값이 어디로부터 왔는지에 대한 정보를 list 형태로 담고 있음
- matrix를 어떻게 채워 나가는지에 대한 정보는 scoring에서 설명

In [125]:
def create_matrix(T1, T2, gap):
    tr_matrix = pd.DataFrame(index=['-']+T2, columns=['-']+T1)
    tr_matrix.iloc[0, 0] = 0
    tr_matrix.iloc[0, 1:] = np.arange(0, len(T1)*gap, gap)
    tr_matrix.iloc[1:, 0] = np.arange(0, len(T2)*gap, gap)
    
    previnfo_matrix = [[[] for j in range(len(T1)+1)] for i in range(len(T2)+1)]
    previnfo_matrix[0][0] = [(0, 0)]
    for y in range(1, len(T1)+1):
        previnfo_matrix[0][y].append((0, y-1))
    for x in range(1, len(T2)+1):
        previnfo_matrix[x][0].append((x-1, 0))
    
    return tr_matrix, previnfo_matrix

In [126]:
mtx, info= create_matrix(T1, T2, -1)
mtx

,-,A,B,C,N,J,R,O,C,L,C,R,P,M
-,0,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12
A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
J,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
J,-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N,-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R,-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C,-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K,-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C,-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Scoring
- 왜 필요한가?
    - 가장 좋은 alignments를 계산하기 위해서
- 가장 좋은 alignments를 계산해야 하는 이유는?
    - 길이가 $l$인 두 traces를 alignment하는 방법의 수는 $\approx(1+\sqrt{2})^{2l+1}l^{-1/2}$ 가지이고, 이 식에 따르면 길이가 100인 두 traces를 alignment하는 방법의 수는 $10^{77}$개나 된다는 결론에 이른다.
        - 왜 그런지 궁금하면 Waterman, M.S.: Introduction to Computational Biology: Maps, sequences and genomes. Chapman & Hall/CRC (2000) 참고
    - 따라서, 이 모든 경우를 비교해서 보여줄 수 없으니 가장 좋은 alignment를 보여주는 것이 필요

### Dynamic programming algorithm
- 위에서 만든 matrix 틀을 채워 나가는 과정
    - 한 줄 한 줄씩 왼쪽에서 오른쪽으로 채워 나간다
- socring 방법은 다음과 같다 (F(i, j): i행 j열에서의 점수)<br><br>
$F(i, j) = max
\begin{cases}
F(i-1,j-1)+S(T_1(i),T_2(j)),\\
F(i, j-1)+gap \,penalty,\\
F(i-1, j)+gap \,penalty.
\end{cases}
$
<br>
<br>
$
S(T_1(i),T_2(j)) = 
\begin{cases}
match \,score &if\, T_1(i)=T_2(j)\\
mismatch \,penalty &else
\end{cases}
$
<br><br><img src="scoring.png", width=500, height=200)><br>
- 주의
    - trace에서 앞에 아무것도 없는 상황에서 insertion 하거나 deletion 하는 것은 (gap이 발생하는 것은) 따로 cost가 없다고 본다.<br> 따라서,
        - 첫 번째 행에서 두 번째 행으로 넘어갈 때 '↓' 방향의 연산은 cost가 0
        - 첫 번째 열에서 두 번째 열로 넘어갈 때 '→' 방향의 연산은 cost가 0
        - 저는 이걸 따로 고려하기 귀찮아서 그냥 처음 matrix 틀 만들 때, 첫 번째 열과 행을 미리 채우고 시작했습니다

In [130]:
def scoring(T1, T2, same=1, diff=-1, gap=-1):
    '''
    smae: activity가 일치할 때 매길 점수, default: 0
    diff: activity가 다를 때 점수, default: -1
    gap: activity를 삽입 하거나 제거할 때, default: -1
    '''
    matrix, prev_info = create_matrix(T1, T2, gap)
    
    for row in range(1, len(matrix.index)):
        
        for col in range(1, len(matrix.columns)):
            
            if row-1 == 0:
                from_top = matrix.iloc[row-1, col]
            else:
                from_top = matrix.iloc[row-1, col] - 1
                
            if col-1 == 0:
                from_left = matrix.iloc[row, col-1]
            else:
                from_left = matrix.iloc[row, col-1] - 1
            
            if matrix.index[row] == matrix.columns[col]:
                from_diagonal = matrix.iloc[row-1, col-1] + 1
            else:
                from_diagonal = matrix.iloc[row-1, col-1] - 1
            
            matrix.iloc[row, col] = max(from_top, from_left, from_diagonal)
            # 화살표 대신 색칠해서 보여주면 될 듯
            if max(from_top, from_left, from_diagonal) == from_top:
                prev_info[row][col].append((row-1, col))
            if max(from_top, from_left, from_diagonal) == from_left:
                prev_info[row][col].append((row, col-1))
            if max(from_top, from_left, from_diagonal) == from_diagonal:
                prev_info[row][col].append((row-1, col-1))
                
    return matrix, prev_info

In [285]:
scored_mtx, info = scoring(T1, T2)
scored_mtx

,-,A,B,C,N,J,R,O,C,L,C,R,P,M
-,0,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12
A,0,1,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11
J,-1,0,0,-1,-2,-1,-2,-3,-4,-5,-6,-7,-8,-9
C,-2,-1,-1,1,0,-1,-2,-3,-2,-3,-4,-5,-6,-7
J,-3,-2,-2,0,0,1,0,-1,-2,-3,-4,-5,-6,-7
N,-4,-3,-3,-1,1,0,0,-1,-2,-3,-4,-5,-6,-7
R,-5,-4,-4,-2,0,0,1,0,-1,-2,-3,-3,-4,-5
C,-6,-5,-5,-3,-1,-1,0,0,1,0,-1,-2,-3,-4
K,-7,-6,-6,-4,-2,-2,-1,-1,0,0,-1,-2,-3,-4
C,-8,-7,-7,-5,-3,-3,-2,-2,0,-1,1,0,-1,-2


## Find path making best alignment
- 위에서 만든 점수표에서 어떤 path를 거쳐야 점수를 높일 수 있는지 시각화
- trace back 과정
    - scoring 때 같이 만든 prev_info를 이용하여 맨 아래 맨 오른쪽 칸에서 시작하여 왼쪽 위 방향으로 그 전 칸이 어디였는지를 찾아 올라감
    <br>
![trace alignment example](https://www.ibm.com/developerworks/library/j-seqalign/LCSTab5.gif)
<br>
- multiple trace인 경우에는 다른 과정이 추가적으로 필요함

In [301]:
def coloring_path(scored_matrix, cells):
    
    def coloring(df):
        frame = df.copy()
        frame.iloc[:, :] = 'background-color: ""'
        frame.iloc[len(scored_matrix.index)-1, len(scored_matrix.columns)-1] = 'background-color: grey'
        for cell in cells:
            frame.iloc[cell[0], cell[1]] = 'background-color: grey'
        return frame
    
    scored_matrix.index = np.arange(len(scored_matrix.index))
    scored_matrix.columns = np.arange(len(scored_matrix.columns))
    
    return scored_matrix.style.apply(coloring, axis=None)

In [302]:
def find_path(prev_info, scored_matrix):
    path = []
    T1_align = []
    T2_align = []
    
    cell = (len(T2), len(T1))
    
    while(cell[0]>0 and cell[1]>0):
        if (cell[0]-1, cell[1]-1) in prev_info[cell[0]][cell[1]]:
            T1_align = [scored_matrix.columns[cell[1]]] + T1_align
            T2_align = [scored_matrix.index[cell[0]]] + T2_align
            path.append((cell[0]-1, cell[1]-1))
            cell = (cell[0]-1, cell[1]-1)
        elif (cell[0], cell[1]-1) in prev_info[cell[0]][cell[1]]:
            T1_align = [scored_matrix.columns[cell[1]]] + T1_align
            T2_align = ['-'] + T2_align
            path.append((cell[0], cell[1]-1))
            cell = (cell[0], cell[1]-1)
        else:
            T1_align = ['-'] + T1_align
            T2_align = [scored_matrix.index[cell[0]]] + T2_align
            path.append((cell[0]-1, cell[1]))
            cell = (cell[0]-1, cell[1])
    
    colored_matrix = coloring_path(scored_matrix.copy(), path)
    
    return T1_align, T2_align, colored_matrix

In [303]:
T1_align, T2_align, colored_mtx = find_path(info, scored_mtx)

print (T1_align)
print (T2_align)
colored_mtx

['A', 'B', 'C', '-', 'N', 'J', 'R', 'O', 'C', 'L', 'C', 'R', '-', 'P', 'M']
['A', 'J', 'C', 'J', 'N', '-', 'R', '-', 'C', 'K', 'C', 'R', 'B', 'P', '-']


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12
1,0,1,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11
2,-1,0,0,-1,-2,-1,-2,-3,-4,-5,-6,-7,-8,-9
3,-2,-1,-1,1,0,-1,-2,-3,-2,-3,-4,-5,-6,-7
4,-3,-2,-2,0,0,1,0,-1,-2,-3,-4,-5,-6,-7
5,-4,-3,-3,-1,1,0,0,-1,-2,-3,-4,-5,-6,-7
6,-5,-4,-4,-2,0,0,1,0,-1,-2,-3,-3,-4,-5
7,-6,-5,-5,-3,-1,-1,0,0,1,0,-1,-2,-3,-4
8,-7,-6,-6,-4,-2,-2,-1,-1,0,0,-1,-2,-3,-4
9,-8,-7,-7,-5,-3,-3,-2,-2,0,-1,1,0,-1,-2


# 예제 데이터

## data import
- 전에 제가 사용했던 데이터를 이용한 것이라 scoring를 하고 나서도 이런 형태의 데이터를 사용하게 될지는 모르겠네요..
- 근데, 일단 뭐라도 대략 어떤 느낌인지 보기 위해 사용하겠습니다.

In [5]:
data = pd.read_csv('Traces.csv')
data

,1_A,1_BAS_4,1_CTTN1_4,1_CXT1_4,1_KL1102_1,1_L7050_1,1_L81281_1,1_L8128_1,1_MGCS_4,1_MGC_4,...,3_X9D00_2,4_A,4_BAS_4,4_CBP0000025_3,4_DM_4,4_FRBY_4,4_KTRI_4,4_NS50V_4,4_PZYC_4,4_RMSI_4
0,1_A,1_BAS_4_7,NaN,1_CXT1_4_7,1_KL1102_1_8,1_L7050_1_7,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,NaN,4_A,4_BAS_4_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1_A,1_BAS_4_7,NaN,1_CXT1_4_7,1_KL1102_1_8,1_L7050_1_7,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,NaN,4_A,4_BAS_4_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1_A,1_BAS_4_7,NaN,1_CXT1_4_7,1_KL1102_1_8,1_L7050_1_7,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,NaN,4_A,4_BAS_4_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1_A,1_BAS_4_7,1_CTTN1_4_7,NaN,1_KL1102_1_8,1_L7050_1_7,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,NaN,4_A,4_BAS_4_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1_A,1_BAS_4_7,1_CTTN1_4_7,NaN,1_KL1102_1_8,1_L7050_1_7,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,NaN,4_A,4_BAS_4_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1_A,1_BAS_4_7,1_CTTN1_4_7,NaN,1_KL1102_1_8,1_L7050_1_7,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,3_X9D00_2_7,4_A,4_BAS_4_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1_A,1_BAS_4_7,1_CTTN1_4_7,NaN,1_KL1102_1_8,1_L7050_1_7,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,3_X9D00_2_7,4_A,4_BAS_4_7,NaN,NaN,4_FRBY_4_0,NaN,NaN,NaN,NaN
7,1_A,1_BAS_4_7,1_CTTN1_4_7,NaN,1_KL1102_1_8,1_L7050_1_8,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,3_X9D00_2_7,4_A,4_BAS_4_7,NaN,NaN,4_FRBY_4_0,NaN,NaN,NaN,NaN
8,1_A,1_BAS_4_7,1_CTTN1_4_7,NaN,1_KL1102_1_8,1_L7050_1_8,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,3_X9D00_2_7,4_A,4_BAS_4_7,NaN,NaN,4_FRBY_4_0,NaN,NaN,NaN,NaN
9,1_A,1_BAS_4_7,1_CTTN1_4_7,NaN,1_KL1102_1_8,1_L7050_1_8,NaN,1_L8128_1_8,NaN,1_MGC_4_7,...,3_X9D00_2_7,4_A,4_BAS_4_7,NaN,NaN,4_FRBY_4_0,NaN,NaN,NaN,NaN


## drawing
- tkinter 모듈 이용
- 각 행, 열의 index를 이용하여 사각형의 위치 설정하고, 
- 각 열의 index를 이용하여 같은 activity의 색 통일하여 그려줌

In [13]:
colors = ['cornflower blue', 'dark slate blue', 'pale green', 'slate blue', 
          'medium slate blue', 'light slate blue', 'blue', 'dodger blue', 
          'gold', 'light goldenrod', 'goldenrod', 'dark goldenrod', 'rosy brown', 
          'indian red', 'saddle brown', 'sandy brown','dark salmon', 'salmon', 
          'light salmon', 'orange', 'dark orange', 'coral', 'light coral', 'tomato', 
          'orange red', 'red', 'hot pink', 'deep pink', 'DarkOrchid1', 'DarkOrchid2', 
          'DarkOrchid3','pale violet red', 'maroon', 'medium violet red', 'violet red', 
          'medium orchid', 'dark orchid', 'dark violet', 'blue violet', 'purple', 
          'medium purple', 'SlateBlue1', 'SlateBlue2', 'SlateBlue3', 'SlateBlue4', 
          'RoyalBlue1', 'RoyalBlue2', 'RoyalBlue3', 'RoyalBlue4', 'blue2','purple1', 
          'purple2', 'purple3', 'purple4']

random.shuffle(colors)

In [33]:
root = Tk()
root.title('Trace alignment')

w = Canvas(root, width=1200, height=600)
w.pack()

for i in range(len(data)):
    w.create_text(50, 50+20*i, text='T%s' % str(i+1))
    
    for j in range(len(data.columns)):
        if data.ix[i, j] is not np.nan:
            w.create_rectangle(70+20*j, 40+20*i, 90+20*j, 60+20*i, fill=colors[j])
        else:
            w.create_rectangle(70+20*j, 40+20*i, 90+20*j, 60+20*i)
            w.create_text(80+20*j, 50+20*i, text='-')
    
mainloop()

- 데이터 양에 따라서 window의 크기도 달라져야 함 (필요에 따라서는 zoom in, out 기능도 필요할 것으로 보임)
- 그림이 나타나는 위치, 그림의 크기는 어떻게 정할 것인지
- 색 조합은 어떻게 할 것인지
- interactive한 환경을 위해서는?
- 더 쉬운 모듈은 없을까